In [3]:
! unzip -q A3_Dataset.zip
! rm A3_Dataset.zip

Preparing dataset folder structure for ImageDataGenerator

In [4]:
! mkdir dataset
! mkdir dataset/train
! mkdir dataset/train/images dataset/train/masks
! mkdir dataset/test
! mkdir dataset/test/images dataset/test/masks

! mv A3_Dataset/dataset1/images_prepped_train dataset/train/images
! mv A3_Dataset/dataset1/images_prepped_test dataset/test/images
! mv A3_Dataset/dataset1/annotations_prepped_train dataset/train/masks
! mv A3_Dataset/dataset1/annotations_prepped_test dataset/test/masks

! rm -r A3_Dataset

Importing packages

In [ ]:
! pip install -U segmentation-models

In [6]:
import numpy as np
import tensorflow as tf

Utility Functions

In [7]:
import matplotlib.pyplot as plt
from skimage import io, color


def show(image):
    io.imshow(image.astype(np.uint8))
    plt.show()

def segshow(mask):
    seg = color.label2rgb(mask, bg_label=0)
    show(seg)

def combinedshow(image, mask_true, mask_pred=None, figsize=(15,3)):
    image = image.astype(np.uint8)
    mask_true = np.argmax(mask_true, 2)
    if mask_pred is not None:
        mask_pred = np.argmax(mask_pred, 2)

    cols = 5 if mask_pred is not None else 3
    fig, ax = plt.subplots(1, cols, figsize=figsize)

    ax[0].set_title('Original Image')
    ax[0].axis('off')
    ax[0].imshow(image)

    ax[1].set_title('Ground Truth')
    ax[1].axis('off')
    ax[1].imshow(color.label2rgb(mask_true, bg_label=0))

    ax[2].set_title('Original + Ground Truth')
    ax[2].axis('off')
    ax[2].imshow(color.label2rgb(mask_true, image, alpha=0.3, bg_label=0))

    if mask_pred is not None:
        ax[3].set_title('Predicted Mask')
        ax[3].axis('off')
        ax[3].imshow(color.label2rgb(mask_pred, bg_label=0))

        ax[4].set_title('Original + Predicted Mask')
        ax[4].axis('off')
        ax[4].imshow(color.label2rgb(mask_pred, image, alpha=0.3, bg_label=0))

    plt.tight_layout()
    plt.show()

Loading dataset

In [8]:
DATASET_DIR = 'dataset/'

TRAIN_X = DATASET_DIR + 'train/images/'
TRAIN_Y = DATASET_DIR + 'train/masks/'

TEST_X = DATASET_DIR + 'test/images/'
TEST_Y = DATASET_DIR + 'test/masks/'

INPUT_SHAPE = (352, 480)
NUM_CLASSES = 12
BATCH_SIZE = 8

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import RandomFlip, RandomCrop, RandomRotation, RandomTranslation, RandomZoom


def load_data(image_dir, mask_dir, num_classes, input_shape, batch_size, train=False):

    def images_masks_generator(images, masks):
        images_masks = zip(images, masks)
        for (imgs, msks) in images_masks:
            rng = np.random.default_rng()

            if train:
                seed = rng.integers(0, 10000)
                imgs = RandomFlip(
                    'horizontal', seed=seed)(imgs).numpy()
                msks = RandomFlip(
                    'horizontal', seed=seed)(msks).numpy()

                seed = rng.integers(0, 10000)
                imgs = RandomCrop(
                    imgs.shape[1], imgs.shape[2], seed=seed)(imgs).numpy()
                msks = RandomCrop(
                    imgs.shape[1], imgs.shape[2], seed=seed)(msks).numpy()

                seed = rng.integers(0, 10000)
                imgs = RandomZoom(
                    0.2, fill_mode='reflect', seed=seed)(imgs).numpy()
                msks = RandomZoom(
                    0.2, fill_mode='reflect', seed=seed)(msks).numpy()

                seed = rng.integers(0, 10000)
                imgs = RandomRotation(
                    0.05, fill_mode='reflect', seed=seed)(imgs).numpy()
                msks = RandomRotation(
                    0.05, fill_mode='reflect', seed=seed)(msks).numpy()

            msks = to_categorical(msks, num_classes=num_classes)

            yield (imgs, msks)

    seed = 42
    validation_split = 0.2 if train else 0.0
    subset = 'training' if train else None
    shuffle = True if train else False

    image_data_generator = ImageDataGenerator(
        validation_split = validation_split)
    
    mask_data_generator = ImageDataGenerator(
        validation_split = validation_split)
    
    images = image_data_generator.flow_from_directory(
        image_dir,
        target_size = input_shape,
        class_mode = None,
        batch_size = batch_size,
        subset = subset,
        shuffle = shuffle,
        seed = seed,
    )
    masks = mask_data_generator.flow_from_directory(
        mask_dir,
        target_size = input_shape,
        color_mode = 'grayscale',
        class_mode = None,
        batch_size = batch_size,
        subset = subset,
        shuffle = shuffle,
        seed = seed,
    )

    if train:
        val_images = image_data_generator.flow_from_directory(
            image_dir,
            target_size = input_shape,
            class_mode = None,
            batch_size = batch_size,
            subset = 'validation',
            shuffle = shuffle,
            seed = seed,
        )
        val_masks = mask_data_generator.flow_from_directory(
            mask_dir,
            target_size = input_shape,
            color_mode = 'grayscale',
            class_mode = None,
            batch_size = batch_size,
            subset = 'validation',
            shuffle = shuffle,
            seed = seed,
        )

        return images_masks_generator(images, masks), images_masks_generator(val_images, val_masks)
    
    return images_masks_generator(images, masks)


train_data, val_data = load_data(TRAIN_X, TRAIN_Y, NUM_CLASSES, INPUT_SHAPE, BATCH_SIZE, train=True)
test_data = load_data(TEST_X, TEST_Y, NUM_CLASSES, INPUT_SHAPE, BATCH_SIZE)

In [ ]:
idx = 0
images, masks = next(val_data)
combinedshow(images[idx], masks[idx])

Model Definition

In [ ]:
import segmentation_models as sm
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

sm.set_framework('tf.keras')
sm.framework()
keras.backend.set_image_data_format('channels_last')

"""
BACKBONE = 'vgg16'
optim = SGD(learning_rate=0.01, momentum=0.9)
"""
BACKBONE = 'resnet50'
optim = Adam(learning_rate=0.001)


def get_model(input_shape, n_classes, backbone):
    input = Input(shape=input_shape)
    preprocessing = sm.get_preprocessing(backbone)
    base_model = sm.Unet(
        backbone_name=backbone,
        input_shape=input_shape,
        classes=n_classes,
        activation='softmax',
        encoder_weights='imagenet',
        decoder_block_type='upsampling')
    base_model.layers.pop(0)

    inputs = input
    x = preprocessing(inputs)
    outputs = base_model(x)

    return Model(inputs, outputs)

model = get_model(INPUT_SHAPE + (3,), NUM_CLASSES, BACKBONE)
model.summary()

Model Training

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


model.compile(
    optim,
    loss=sm.losses.categorical_focal_dice_loss,
    metrics=['accuracy',
             sm.metrics.iou_score,
             sm.metrics.f1_score,
             sm.metrics.precision,
             sm.metrics.recall],
)

callbacks = [
    ReduceLROnPlateau(
        factor=0.1, min_delta=0.01, min_lr=0.00001, patience=5, verbose=1),
    EarlyStopping(
        min_delta=0.001, patience=10, restore_best_weights=True, verbose=1)
]

H = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    steps_per_epoch=np.ceil(294 / BATCH_SIZE),
    validation_steps=np.ceil(73 / BATCH_SIZE),
    callbacks=callbacks,
)

Training Plots

In [ ]:
plt.figure(figsize=(6, 4))
plt.title("Categorical Focal Loss + Jaccard Loss")
plt.plot(H.history["loss"], label="Training")
plt.plot(H.history["val_loss"], label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc='upper right')
plt.show()

plt.figure(figsize=(6, 4))
plt.title("Accuracy")
plt.plot(H.history["accuracy"], label="Training")
plt.plot(H.history["val_accuracy"], label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(loc='lower right')
plt.show()

plt.figure(figsize=(6, 4))
plt.title("Jaccard Score")
plt.plot(H.history["iou_score"], label="Training")
plt.plot(H.history["val_iou_score"], label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Score")
plt.legend(loc='upper left')
plt.show()

plt.figure(figsize=(6, 4))
plt.title("Dice Score")
plt.plot(H.history["f1-score"], label="Training")
plt.plot(H.history["val_iou_score"], label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Score")
plt.legend(loc='upper left')
plt.show()


Quantitative Evaluation

In [ ]:
scores = model.evaluate(
    test_data,
    steps = np.ceil(101 / BATCH_SIZE),
    return_dict = True,
)

print('Jaccard Score: {:.2f}'.format(scores['iou_score']))
print('Dice Score: {:.2f}'.format(scores['f1-score']))
print('Accuracy: {:.2f}'.format(scores['accuracy']))
print('Precision: {:.2f}'.format(scores['precision']))
print('Recall: {:.2f}'.format(scores['recall']))

Qualitative Evaluation

In [ ]:
for i in range(4):
    images, masks = next(test_data)
    image, mask = images[-1], masks[-1]
    pred = model(np.array([image]))[0]
    combinedshow(image, mask, pred)